In [1]:
import sys,os
sys.path.append('..')
from ccat_holo.Pyccat import CCAT_holo
import time
import torch as T
import numpy as np
import h5py


from ccat_holo.fitting_error_plot import Fit_M_Surface,Fit_M_Surface_zk,colormap
from ccat_holo.pyplot import plot_beamcontour
from ccat_holo.zernike_torch import N as poly_N

In [2]:
#dz_Rx=715
# 1. Model folder
Model_folder='../CCAT_model2'
# 2. Configuration of the Holography system
holo_setup={'Rx1':([0,0,600],'../scan/mainbeam/on-axis.txt'),
            #'Rx2':([400,400,600],'../scan/mainbeam/pos_pos_near.txt'),
            #'Rx3':([400,-400,600],'../scan/mainbeam/pos_neg_near.txt'),
            #'Rx4':([-400,400,600],'../scan/mainbeam/neg_pos_near.txt'),
            #'Rx5':([-400,-400,600],'../scan/mainbeam/neg_neg_near.txt')
            }
# Define 5 receiver positions in the 'Coord_Rx' system, e.g [+/-400,+/-400,600]
# and their field points that are stored in file 'scan/400_400_600.txt'. The field
# points of the measured beam maps are from the read out of the telescope coders.
#####################
# 3. Output_folder is the folder used to store the intermediate data of the first
#    beam calculation and the analysing results.
#Output_folder='Analysis_beam'
Output_folder='test_square'
# 4. Create the FYST holography Model and check the telescope model and 
# holographic setup in the 3D view.
Model=CCAT_holo(Model_folder,Output_folder,holo_conf=holo_setup)

FYST telescope model has been created!!


In [3]:
Model.view() # Show 3D FYST model and its receivers.
Model.view_Rx(Rx=['Rx2']) # This method can highlight the chosen reciever horns.

[-1.2490457723982542, 0, 0] [0, -4799.999999999999, 0]


Widget(value='<iframe src="http://localhost:62438/index.html?ui=P_0x1922a2ee6d0_0&reconnect=auto" class="pyvis…

In [7]:
# 5. Running the time-consuming first beam calculation.
Model.First_Beam_cal(S2_init=np.zeros(5),S1_init=np.zeros(5))

# 6. Make the farward beam calculation function, (Model.FF)
z_order=7
#Model.mk_FF(fitting_param='zernike',Device=T.device('cuda'),Z_order=z_order)

The holographic setup:
Rx1 : [0, 0, 600] ../scan/mainbeam/on-axis.txt

***Start the initial beam calculations 
***and prepare the required Matrixes used to speed up the forward beam calculations.
Rx1 : [0, 0, 600] ../scan/mainbeam/on-axis.txt
step 1:

step 2:

step 3:

step 4:

time used: 698.0073993999999


In [5]:
np.sqrt(1552500)

1245.9935794377113

In [5]:
for n in range(10):
    Model.FF(T.zeros((2,poly_N(z_order))).to(T.device('cuda')),
                  T.zeros(5*6),
                  T.zeros(5*5)).cpu().numpy()

In [7]:
# 7. load the meausred beams
'''Meaused data'''
Meas_beam=np.genfromtxt('./Meas_beams/GRASP_296GHz_31_31_0_23Twist_10umPanel.txt',delimiter=',')
Meas_beam=T.tensor(Meas_beam).to(T.device('cuda'))
Power = T.sum(T.sum(Meas_beam.reshape(5,2,-1)**2,axis=-2),axis=-1).repeat(2).reshape(5*2,-1)
for n in range(10):
    Meas_beam[n,:] = Meas_beam[n,:] + \
        T.tensor(np.random.normal(0,np.sqrt(Power[n].cpu().numpy())*10**(-50/20),
                                  Meas_beam.size()[1]),
                                  dtype=T.float64,device =T.device('cuda'))

In [ ]:
# 8. Find the large-scale parameters. The fitting results are stored in file 'fit_LP.h5py'
Model.fit_LP(Meas_beam,Device=T.device('cuda'),outputfilename='fit_LP_zk_0_m2m1',Init=np.zeros((2,poly_N(z_order))))

In [9]:
# Load the results from h5py file
with h5py.File(Output_folder+'/fit_LP_zk_0_m2m1.h5','r') as f:
    fit_LP=f['x'][:]

In [ ]:
fit_LP[5*6:].reshape(5,-1)

In [ ]:
Beams_LP=Model.FF(T.zeros((2,poly_N(z_order))).to(T.device('cuda')),
                  fit_LP[0:5*6],
                  fit_LP[5*6:]).cpu().numpy()
Beams_ref=Model.FF(T.zeros((2,poly_N(z_order))).to(T.device('cuda')),
                  T.tensor([1,0,0,0,0,0,
                            1,0,0,0,0,0,
                            1,0,0,0,0,0,
                            1,0,0,0,0,0,
                            1,0,0,0,0,0]),
                  T.zeros(5*5)).cpu().numpy()
# Blue contour lines represent reference beams.
# Red is the fitted beams for the measured beams. 
x0=np.linspace(-1,1,31)
y0=np.linspace(-1,1,31)
plot_beamcontour(x0,y0,Beams_LP,Meas_beam.cpu().numpy(),#Beams_ref,
                 levels=[-35,-30,-20,-15])

In [ ]:
# 9. analysis the surface errors
Model.fit_surface_zk(Meas_beam,
                  constraint=[0.1,0.1],
                  Device=T.device('cuda'),
                  Init_LP=fit_LP,
                  outputfilename='fit_adjusters_zernike_0_m2m1')

In [13]:
# Load the results from h5py file
with h5py.File(Output_folder+'/fit_adjusters_zernike_0_m2m1.h5','r') as f:
    fit_zk=f['x'][:]

In [ ]:
Fit_M_Surface_zk(fit_zk[0:-5*11],z_order,model_file='../CCAT_model',vmax=50,vmin=-50)

In [ ]:
Beams_zk=Model.FF(T.cat((T.zeros((2,1)),T.tensor(fit_zk[0:-5*11]).reshape(2,-1)),axis=1).to(T.device('cuda')),
                  T.tensor(fit_zk[-5*11:-5*5]).to(T.device('cuda')),
                  T.tensor(fit_zk[-5*5:]).to(T.device('cuda'))).cpu().numpy()
Beams_ref=Model.FF(T.zeros((2,poly_N(z_order))).to(T.device('cuda')),
                  T.tensor([1,0,0,0,0,0,
                            1,0,0,0,0,0,
                            1,0,0,0,0,0,
                            1,0,0,0,0,0,
                            1,0,0,0,0,0]).to(T.device('cuda')),
                  T.zeros(5*5).to(T.device('cuda'))).cpu().numpy()
# Blue contour lines represent reference beams.
# Red is the fitted beams for the measured beams. 
x0=np.linspace(-1,1,31)
y0=np.linspace(-1,1,31)
plot_beamcontour(x0,y0,Beams_zk,Meas_beam.cpu().numpy(),
                 levels=[-35,-30,-20,-15])

In [ ]:
# 9. analysis the surface errors
Model.fit_surface_zk_Amp(Meas_beam,
                  constraint=[1,1],
                  Device=T.device('cuda'),
                  Init_LP=fit_LP,
                  outputfilename='fit_adjusters_zernike_0_AMP_m2m1')


In [17]:
# Load the results from h5py file
with h5py.File(Output_folder+'/fit_adjusters_zernike_0_AMP_m2m1.h5','r') as f:
    fit_zk_AMP=f['x'][:]

In [ ]:
Fit_M_Surface_zk(fit_zk_AMP[0:-5*11],z_order,model_file='../CCAT_model',vmax=100,vmin=-100)

In [ ]:
Beams_zk=Model.FF(T.cat((T.zeros((2,1)),T.tensor(fit_zk_AMP[0:-5*11]).reshape(2,-1)),axis=1).to(T.device('cuda')),
                  T.tensor(fit_zk_AMP[-5*11:-5*5]).to(T.device('cuda')),
                  T.tensor(fit_zk_AMP[-5*5:]).to(T.device('cuda'))).cpu().numpy()
Beams_ref=Model.FF(T.zeros((2,poly_N(z_order))).to(T.device('cuda')),
                  T.tensor([1,0,0,0,0,0,
                            1,0,0,0,0,0,
                            1,0,0,0,0,0,
                            1,0,0,0,0,0,
                            1,0,0,0,0,0]).to(T.device('cuda')),
                  T.zeros(5*5).to(T.device('cuda'))).cpu().numpy()
# Blue contour lines represent reference beams.
# Red is the fitted beams for the measured beams. 
x0=np.linspace(-1,1,31)
y0=np.linspace(-1,1,31)
plot_beamcontour(x0,y0,Beams_zk,Meas_beam.cpu().numpy(),
                 levels=[-35,-30,-20,-15])